In [ ]:
import cv2
import numpy as np
from openvino.inference_engine import IECore

# Load the Inference Engine
ie = IECore()

# Load the optimized model (SSDLite MobileNetV2)
model_xml = 'ssdlite_mobilenet_v2/frozen_inference_graph.xml'
model_bin = 'ssdlite_mobilenet_v2/frozen_inference_graph.bin'
net = ie.read_network(model=model_xml, weights=model_bin)
exec_net = ie.load_network(network=net, device_name='CPU')

# Load class labels for the model
with open('ssdlite_mobilenet_v2/ssdlite_mobilenet_v2.labels', 'r') as f:
    labels = f.read().strip().split('\n')

# Open a video capture source (webcam in this case)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Perform inference on the frame
    input_blob = next(iter(net.inputs))
    output_blob = next(iter(net.outputs))
    frame_input = cv2.resize(frame, (300, 300))  # Resize input image to match the model's input size
    frame_input = frame_input.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    frame_input = frame_input.reshape((1, 3, 300, 300))  # Add batch dimension
    res = exec_net.infer(inputs={input_blob: frame_input})

    # Process the output to draw bounding boxes on the frame
    detections = res[output_blob][0]
    for detection in detections:
        confidence = detection[2]
        if confidence > 0.5:  # Filter out low-confidence detections
            class_id = int(detection[1])
            label = labels[class_id]
            x1, y1, x2, y2 = (detection[3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])).astype(int)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label}: {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with bounding boxes
    cv2.imshow('Object Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
